In [1]:
'''Linear algebra'''
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

'''Data visualisation'''
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot # cf. 5.2

'''Preprocessing'''
from sklearn.impute import SimpleImputer # cf. 5.2
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,LabelEncoder,OneHotEncoder # cf. 5.2

'''Deep learning'''
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, RNN, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences, TimeseriesGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

'''Pipeline'''
from sklearn.pipeline import Pipeline,FeatureUnion # cf. 5.6
from sklearn.compose import ColumnTransformer,make_column_selector # cf. 5.6
from sklearn.preprocessing import FunctionTransformer # cf. 5.6
from sklearn.base import TransformerMixin,BaseEstimator # cf. 5.6

# Data preprocessing

## Loading dataset

In [2]:
dataset = pd.read_csv('../raw_data/ytrain_raw.csv')

/Users/nhuyghe/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dataset_cleaned = dataset.fillna('Down')
dataset_cleaned['timestamp'] = pd.to_datetime(dataset_cleaned['timestamp'])
dataset_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31232 entries, 0 to 31231
Columns: 274 entries, timestamp to S99-T2
dtypes: datetime64[ns, UTC](1), object(273)
memory usage: 65.3+ MB


In [4]:
dataset_cleaned = dataset.set_index('timestamp').fillna('Down')
dataset_cleaned = dataset_cleaned[:10000]

In [5]:
for col in dataset_cleaned.columns:
    dataset_cleaned[col] = dataset_cleaned[col].map({'Available':0, 'Charging':1, 'Down':0, 'Offline':0, 'Passive':0})

In [6]:
# Rename columns
borns_name = list(dataset_cleaned.columns)
stations_name = list(dataset_cleaned.columns.map(lambda x:x[:-3]))
cols_dict = {borns_name[i]: stations_name[i] for i in range(len(borns_name))}

df_stations = dataset_cleaned.rename(columns=cols_dict)

In [7]:
df_stations = df_stations.groupby(level=0,axis=1).sum()

In [8]:
df_stations.head()

,S1,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S2,S20,S21,S22,S23,S24,S25,S26,S27,S28,S29,S3,S30,S31,S32,S33,S34,S35,S36,S37,S38,S39,S4,S41,S42,S45,S46,S47,S48,S49,S5,S50,S51,S52,S53,S56,S57,S58,S59,S6,S60,S62,S63,S64,S65,S66,S67,S68,S69,S7,S70,S71,S72,S74,S75,S76,S77,S78,S79,S8,S80,S81,S82,S83,S84,S85,S86,S87,S88,S89,S9,S91,S92,S93,S94,S95,S96,S97,S98,S99
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-25T00:00:00+00:00,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,3,0,1,0,0,0,2,0,0,0,0,0,2,0,0,1,0,0,0,0,0,1,1,0,0,0,0,2,0,0,0,2,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2019-11-25T00:15:00+00:00,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,0,2,0,0,1,0,0,1,0,0,1,1,0,0,0,1,2,0,0,0,2,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2019-11-25T00:30:00+00:00,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2019-11-25T00:45:00+00:00,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,2,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2019-11-25T01:00:00+00:00,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,3,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,2,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


## Clustering

In [9]:
X_train = np.expand_dims(df_stations.to_numpy().T,axis=-1)

In [10]:
print(X_train.shape)

(91, 10000, 1)


In [ ]:
%%time
from tslearn.clustering import TimeSeriesKMeans
km = TimeSeriesKMeans(n_clusters=5, metric="softdtw")
labels = km.fit_predict(X_train)
labels

In [ ]:
df_clustering = df_stations.T
df_clustering

In [ ]:
df_clustering['cluster_group'] = labels
df_clustering

In [ ]:
df_clustering.groupby('cluster_group').mean().T.plot()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
df_clustering.groupby('cluster_group').count()